# Building Classifiers

In [2]:
import Classification_Utils as cu
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## Binary flag determines whether to convert data to 1 and 0 for presence/absence

In [2]:
BINARY = False

## Load and combine data from all tissues

In [11]:
df = pd.read_csv('FullPeptideQuant.txt', sep='\t', index_col='Peptide')
print(df.shape)

KeyboardInterrupt: 

In [4]:
if BINARY:
    df = cu.abundance_to_binary(df)
    
df.head()

,Blood_Plasma_CPTAC_TrypDige_undepleted_normal_19Apr13_Methow_13-02-13,Blood_Plasma_Darpa_2_human_02_23Jan17_Arwem_16-10-25,Blood_Plasma_OMICS_EBV_HP_UW001_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW002_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW003_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW004_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW005_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW006_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW007_8Apr16_Arwen_16-01-03,Blood_Plasma_OMICS_EBV_HP_UW008_8Apr16_Arwen_16-01-03,...,Temporal_Lobe_Alz_FX1P159_Guan_1_26Jul10_Andromeda_10-06-28,Temporal_Lobe_Alz_FX1P159_Guan_2_26Jul10_Andromeda_10-06-29,Temporal_Lobe_Alz_FX1P159_Guan_3_26Jul10_Andromeda_10-06-28,Temporal_Lobe_Alz_FX1P159_Guan_Typx2_1_26Jul10_Andromeda_10-06-29,Temporal_Lobe_Alz_FX1P159_Guan_Typx2_2_26Jul10_Andromeda_10-06-28,Temporal_Lobe_Alz_FX1P159_Guan_Typx2_3_26Jul10_Andromeda_10-06-29,Temporal_Lobe_Alz_FX2P57_IMAC_153_9May11_Hawk_11-04-02p,Temporal_Lobe_Alz_FX2P57_IMAC_161_20Apr11_Hawk_10-12-03p,Temporal_Lobe_Alz_FX2P57_IMAC_187_26Apr11_Hawk_10-12-03p,Temporal_Lobe_Alz_FX2P57_IMAC_212_11May11_Hawk_11-04-02p
Peptide,,,,,,,,,,,,,,,,,,,,,
-.DIQM*TQSPSTLSASVGDR.V,26.731951,22.187643,3.022208,29.328345,29.916272,3.022208,27.586706,30.458361,29.004130,31.023004,...,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208
-.DIQM*TQSPSTLSASVGDRVTITCR.A,3.022208,3.022208,3.022208,30.633308,30.815586,3.022208,29.483431,32.564995,30.319263,32.368436,...,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208
-.DIQMTQSPSTLSASVGDR.V,26.387537,28.015792,3.022208,3.022208,31.403752,3.022208,27.695976,29.779972,29.747784,30.255299,...,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208
-.DIQMTQSPSTLSASVGDRVTITCR.A,3.022208,3.022208,3.022208,3.022208,32.585511,3.022208,3.022208,30.845879,30.983525,27.526416,...,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208
-.EVQLVETGGGLIQPGGSLR.L,24.546220,3.022208,3.022208,3.022208,26.732727,3.022208,28.163126,23.836245,3.022208,28.436388,...,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208,3.022208


## Map each column to a corresponding label

In [12]:
tissues = ['Blood_Plasma', 'Blood_Serum', 'CSF', 'Liver', 'Monocyte', 'Ovary', 'Pancreas', 'Substantia_Nigra', 'Temporal_Lobe']
 
tissues_to_columns = cu.map_tissues_to_columns(df, tissues)

In [13]:
column_names = df.columns.values.tolist()
labels = cu.get_labels(column_names, tissues_to_columns)

## Make train-test split

In [7]:
train_df, test_df, train_labels, test_labels = train_test_split(
    df.T, labels, 
    test_size=0.30,    # 30% of the data held out in test set
    random_state=0,    # Setting random_state ensures the same train/test split occurs each time this is run
    stratify=labels)   # Maintain ratio of tissues represented in each set

train_features = train_df.columns.values.tolist()

In [8]:
if BINARY:
    min_max_train_df = train_df
    min_max_test_df = test_df
else:
    std_scaler = StandardScaler()
    train_df = std_scaler.fit_transform(train_df)
    test_df = std_scaler.fit_transform(test_df)

    min_max_scaler = MinMaxScaler()
    min_max_train_df = min_max_scaler.fit_transform(train_df)
    min_max_test_df = min_max_scaler.fit_transform(test_df)

## Train various classifiers, using cross-validation to produce an accuracy score (Supplementary Figure XXX)

In [9]:
NUM_SPLITS = 100 # number of train/test splits in cross validation

In [10]:
import time

### KNN

In [ ]:
start = time.time()
knn = cu.knn_model_crossval(train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

accuracy: 0.81 (+/- 0.10)
Runtime: 1.5255493680636087 minutes


### Logistic Regression

In [11]:
start = time.time()
lr = cu.logistic_regression_model_crossval(train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

accuracy: 0.99 (+/- 0.03)
Runtime: 55.156165635585786 minutes


### Naive Bayes
* Gaussian
* Multinomial

In [12]:
start = time.time()
gnb = cu.bayes_gaussian_model_crossval(train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

accuracy: 0.97 (+/- 0.05)
Runtime: 0.8184105157852173 minutes


In [13]:
start = time.time()
mnb = cu.bayes_multinomial_model_crossval(min_max_train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

accuracy: 0.95 (+/- 0.06)
Runtime: 0.1397305687268575 minutes


### SVC 

In [17]:
start = time.time()
svc = cu.SVC_model_crossval(train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

KeyboardInterrupt: 

### Aggregations
* Random Forest
* Gradient Boosting

In [ ]:
start = time.time()
rf = cu.randomforest_model_crossval(train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

In [ ]:
start = time.time()
gbc = cu.gradient_boosting_crossval(train_df, train_labels, NUM_SPLITS)
end = time.time()
print("Runtime:", (end - start)/60, "minutes")

## Classify Training Set

### Use models from notebook to predict new data

In [ ]:
lr_pred = lr.predict(test_df)
lr_result = lr.score(test_df, test_labels)

mnb_pred = mnb.predict(min_max_test_df)
mnb_result = mnb.score(test_df, test_labels)

rf_pred = rf.predict(test_df)
rf_result = rf.score(test_df, test_labels)

svc_pred = svc.predict(test_df)
svc_result = svc.score(test_df, test_labels)

gbc_pred = gbc.predict(test_df)
gbc_result = gbc.score(test_df, test_labels)

gnb_pred = gnb.predict(test_df)
gnb_result = gnb.score(test_df, test_labels)

knn_pred = knn.predict(test_df)
knn_result = knn.score(test_df, test_labels)

In [ ]:
print(lr_result)
print(mnb_result)
print(rf_result)
print(svc_result)
print(gbc_result)
print(gnb_result)
print(knn_result)

##  Confusion matrices of model's predictions on new data

In [ ]:
cm_labels = list(set(gnb_pred.tolist() + test_labels))

cu.show_confusion_matrices(test_labels, gnb_pred, cm_labels)

## Classify Liver Cell Line Data

In [23]:
cell_line_df = pd.read_csv('TrainTestCellLineQuant.txt', sep='\t', index_col='Peptide')
cell_line_df = cell_line_df.filter(like='Cell_Line', axis=1) # Break off cell line data

cell_line_df = cell_line_df.T[train_features]
cell_line_df = std_scaler.fit_transform(cell_line_df)
min_max_cell_line_df = min_max_scaler.fit_transform(cell_line_df)

cell_line_labels = ['Liver' for i in range(10)]
cell_line_df.shape

NameError: name 'train_features' is not defined

In [ ]:
lr_cell_line_pred = lr.predict(cell_line_df)
lr_cell_line_result = lr.score(cell_line_df, cell_line_labels)

mnb_cell_line_pred = mnb.predict(min_max_cell_line_df)
mnb_cell_line_result = mnb.score(min_max_cell_line_df, cell_line_labels)

rf_cell_line_pred = rf.predict(cell_line_df)
rf_cell_line_result = rf.score(cell_line_df, cell_line_labels)

svc_cell_line_pred = svc.predict(cell_line_df)
svc_cell_line_result = svc.score(cell_line_df, cell_line_labels)

gbc_cell_line_pred = gbc.predict(cell_line_df)
gbc_cell_line_result = gbc.score(cell_line_df, cell_line_labels)

gnb_cell_line_pred = gnb.predict(cell_line_df)
gnb_cell_line_result = gnb.score(cell_line_df, cell_line_labels)

knn_cell_line_pred = knn.predict(cell_line_df)
knn_cell_line_result = knn.score(cell_line_df, cell_line_labels)

In [ ]:
print(lr_cell_line_result)
print(mnb_cell_line_result)
print(rf_cell_line_result)
print(svc_cell_line_result)
print(gbc_cell_line_result)
print(gnb_cell_line_result)
print(knn_cell_line_result)

In [ ]:
print(lr_cell_line_pred)
print(mnb_cell_line_pred)
print(rf_cell_line_pred)
print(svc_cell_line_pred)
print(gbc_cell_line_pred)
print(gnb_cell_line_pred)
print(knn_cell_line_pred)

##  Confusion matrices of cell line predictions 

In [ ]:
cellline_cm_labels = list(set(['Liver'] + lr_cell_line_pred.tolist()))

cu.show_confusion_matrices(cell_line_labels, lr_cell_line_pred, cellline_cm_labels)

## Compare peptides in Liver to Liver Cell Line

In [17]:
tissue_abundances = cu.get_descending_abundances(df, labels)

In [24]:
cell_line_labels = ['Cell_Line' for i in range(10)]
cell_line_abundances = cu.get_descending_abundances(cell_line_df, cell_line_labels)

In [25]:
top_50_liver_peptides = cu.n_most_abundant(tissue_abundances, 'Liver', 50)
top_50_cell_line_peptides = cu.n_most_abundant(cell_line_abundances, 'Cell_Line', 50)

In [27]:
peptide_overlap = list(set(top_50_liver_peptides) & set(top_50_cell_line_peptides))

In [33]:
top_50_liver_peptides[:10]

array(['R.VAPEEHPVLLTEAPLNPK.A', 'K.VLGAFSDGLAHLDNLK.G',
       'K.SYELPDGQVITIGNER.F', 'K.EFTPPVQAAYQK.V', 'K.ELGATECINPQDYK.K',
       'K.VIPLFTPQCGK.C', 'R.AVFPSIVGRPR.H', 'R.SIFSAVLDELK.V',
       'R.GALQNIIPASTGAAK.A', 'K.LVNEVTEFAK.T'], dtype=object)

In [32]:
top_50_cell_line_peptides[:10]

array(['R.VTIAQGGVLPNIQAVLLPK.K', 'R.VAPEEHPVLLTEAPLNPK.A',
       'K.AMGIMNSFVNDIFER.I', 'K.SYELPDGQVITIGNER.F', 'R.AGLQFPVGR.I',
       'K.HLVDEPQNLIK.Q', 'K.LVNELTEFAK.T', 'K.GLVLIAFSQYLQQCPFDEHVK.L',
       'R.TTGIVMDSGDGVTHTVPIYEGYALPHAILR.L', 'R.LGEDNINVVEGNEQFISASK.S'], dtype=object)

In [30]:
peptide_overlap

['K.KVPQVSTPTLVEVSR.N',
 'R.VPTANVSVVDLTCR.L',
 'R.GYSFTTTAER.E',
 'K.SYELPDGQVITIGNER.F',
 'K.YPIEHGIITNWDDMEK.I',
 'K.IISNASCTTNCLAPLAK.V',
 'R.GALQNIIPASTGAAK.A',
 'R.TTGIVMDSGDGVTHTVPIYEGYALPHAILR.L',
 'R.VAPEEHPVLLTEAPLNPK.A']